In [4]:
import psycopg2
import nltk
import pandas as pd
import numpy as np
import re
import string
import spacy
import sklearn
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

In [5]:
# importing and creating df (has to have type_id)

# usage: specify file location, sample size and seed(used by random) 
filepath = '/Users/Master/Documents/KU/2.Semester/Datascience/news_sample.csv' #
#s = 250                                            # desired sample size
#seed = 1                                           # seed used by Pseudorandom number generator

df = pd.read_csv(filepath, index_col = [0])
content = df['content']

In [38]:
df['content']

0      Sometimes the power of Christmas will make you...
1      AWAKENING OF 12 STRANDS of DNA – “Reconnecting...
2      Never Hike Alone: A Friday the 13th Fan Film U...
3      When a rare shark was caught, scientists were ...
4      Donald Trump has the unnerving ability to abil...
                             ...                        
245    Prison for Rahm, God’s Work And Many Others\n\...
246    4 Useful Items for Your Tiny Home\n\nHeadline:...
247    Former CIA Director Michael Hayden said Thursd...
248    Antonio Sabato Jr. says Hollywood's liberal el...
249    Former U.S. President Bill Clinton on Monday c...
Name: content, Length: 250, dtype: object

In [6]:
nlp = spacy.load('en_core_web_lg')

In [23]:
text = ("these word are features")
with nlp.disable_pipes():
    vectors = np.array([token.vector for token in nlp(text)])

In [7]:
doc = nlp("these word, are features")
for token in doc:
    print(token)
#token.lemma_
#token.is_stop

these
word
,
are
features


In [4]:
with nlp.disable_pipes():
    doc_vectors = np.array([nlp(sentence).vector for sentence in df['content']])

In [66]:
fakeidx = df.index[df.type == 'fake'].tolist()

In [11]:
fakevecs = np.array([doc_vectors[i] for i in fakeidx])

In [54]:

len(fakevecs)
np.linalg.norm(fakevecs[0])
#calc avg vector of the type-vectors:
#np.linalg.norm(fakevecs[29])

2.442973

In [8]:
df.type.fillna('no type', inplace = True)
#X_train, X_test, y_train, y_test = train_test_split(doc_vectors, df.type, test_size = 0.1, random_state= 1)

In [28]:
a = nlp('REPLY NOW FOR FREE TEA').vector
b = nlp('REPLY NOW FOR FREE TEA AND Potatoes').vector

In [16]:
svc = LinearSVC(random_state = 1, dual = False, max_iter = 10000)
svc.fit(X_train, y_train)
print(f"Accuracy: {svc.score(X_test, y_test) * 100:.3f}%",)
#svc.score(X_test,y_test) * 100

Accuracy: 64.000%


In [29]:
#sklearn.metrics.pairwise.cosine_similarity(a,b)
def cosine_similarity(a,b):
    return a.dot(b)/np.sqrt(a.dot(a) * b.dot(b))

In [37]:
cosine_similarity(fakevecs[152],doc_vectors[45])

0.95132565

In [7]:
#try tf-idf svm
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
x = v.fit_transform(df["content"])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(x, df.type, test_size = 0.1, random_state= 1)

In [12]:
from sklearn import svm
clf = svm.SVC(gamma='scale')
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.52

In [80]:
x = nlp("man")
y = nlp("king")
print(x.similarity(y))
print(x.similarity(x))



0.40884613016820903
1.0


In [7]:
import gensim 
from gensim.models.doc2vec import TaggedDocument, Doc2Vec 
from collections import namedtuple

In [9]:
df['type_id'] = df.groupby(['type']).ngroup()

In [138]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
series_content = df.content

contTok =series_content.str.split().tolist()
typeLst =df.type_id.tolist()

doc1 = ["This is a sentence", "This is another sentence"]
docs = []
analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
for i, text in enumerate(doc1):
    words = text.lower().split()
    tags = [i]
    docs.append(analyzedDocument(words, tags))

['Sometimes',
 'the',
 'power',
 'of',
 'Christmas',
 'will',
 'make',
 'you',
 'do',
 'wild',
 'and',
 'wonderful',
 'things.',
 'You',
 'do',
 'not',
 'need',
 'to',
 'believe',
 'in',
 'the',
 'Holy',
 'Trinity',
 'to',
 'believe',
 'in',
 'the',
 'positive',
 'power',
 'of',
 'doing',
 'good',
 'for',
 'others.',
 'The',
 'simple',
 'act',
 'of',
 'giving',
 'without',
 'receiving',
 'is',
 'lost',
 'on',
 'many',
 'of',
 'us',
 'these',
 'days,',
 'as',
 'worries',
 'about',
 'money',
 'and',
 'success',
 'hold',
 'us',
 'back',
 'from',
 'giving',
 'to',
 'others',
 'who',
 'are',
 'in',
 'need.',
 'One',
 'congregation',
 'in',
 'Ohio',
 'was',
 'moved',
 'to',
 'action',
 'by',
 'the',
 'power',
 'of',
 'a',
 'sermon',
 'given',
 'at',
 'their',
 'church',
 'on',
 'Christmas',
 'Eve.',
 'The',
 'pastor',
 'at',
 'Grand',
 'Lake',
 'United',
 'Methodist',
 'Church',
 'in',
 'Celina,',
 'Ohio',
 'gave',
 'an',
 'emotional',
 'sermon',
 'about',
 'the',
 'importance',
 'of',
 'und

In [17]:
#model = Doc2Vec(docs, size = 100, window = 300, min_count = 1, workers = 4)

In [98]:
#model = gensim.models.doc2vec.Doc2Vec(vector_size=250, min_count=2, epochs=40)
#documents = [TaggedDocument(contTok,df.index) for sentence in df]
documents = [TaggedDocument(contTok, [i]) for i, contTok in enumerate(contTok)]

In [139]:
documents

was', 'utterly', 'nothing', 'objectionable', 'in', 'any', 'way,', 'shape', 'or', 'form', 'in', 'that', 'video,', 'but', 'since', 'she', 'was', 'out', 'of', 'town', 'and', 'I', 'noted', 'the', 'lack', 'of', 'ads', 'I', 'filed', 'the', 'protest', 'for', 'her.', 'They', 'turned', 'it', 'back', 'on.', 'Instead', 'of', 'looking', 'before', 'killing', 'it', 'in', 'the', 'first', 'place,', 'of', 'course,', 'they', 'just', 'used', 'a', 'computer', 'to', 'do', 'it,', 'you', 'see,', 'because', '30', 'seconds', 'of', 'actual', 'looking', 'would', 'have', 'disclosed…..', 'a', 'cat', 'and', 'a', 'hike.', 'It', 'also', 'would', 'have', 'meant', 'Google', 'would', 'have', 'made', 'a', 'bit', 'less', 'money', 'to', 'actually', 'employ', 'people,', 'and,', 'well,', 'we', 'can’t', 'have', 'that.', 'Jobs?', 'Forget', 'about', 'it.', 'Then', 'there’s', 'Chicago', 'and', 'Goldman’s', 'latest', 'fraud,', 'and', 'what', 'a', 'doozey', 'that', 'one', 'is.', 'The', 'City,', 'which', 'incidentally', 'is', 'basi

In [99]:
model = Doc2Vec(documents, vector_size = 250, min_count = 2, epochs= 40)

In [100]:
vector = model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])
#doc2 = nlp(df.content[20])
#tokdoc2 = [token for token in nlp(df.content[20])] 
#for token in doc2:
#    print(token)

In [141]:
v = model.infer_vector(contTok[25])

In [193]:
sim_doc = model.docvecs.most_similar([12])
sim_doc

[(196, 0.8644919395446777),
 (83, 0.817737340927124),
 (239, 0.7764697074890137),
 (4, 0.7751911878585815),
 (128, 0.7693681716918945),
 (192, 0.7382450699806213),
 (15, 0.7334905862808228),
 (110, 0.7207229137420654),
 (74, 0.7105898857116699),
 (66, 0.6957017183303833)]

In [198]:
df.type[128]

'fake'

In [192]:
df.loc[df.type == 'bias']

,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,type_id
12,1685,charismanews.com,bias,https://charismanews.com/politics/69268-in-cal...,"President Trump, right, with Rep. Robert Light...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"In Calling Out 'Fake News,' Donald Trump Engag...",Hamilton Strategies,NaN,"['fake news', 'donald trump', 'president trump...","Stephen E. Strang, author of 'God and Donald T...",NaN,NaN,0
114,19639,unz.com,bias,http://www.unz.com/author/anatoly-karlin/topic...,Here I outline one of the core philosophies of...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Anatoly Karlin Archive,"Anatoly Karlin, Sayed Hasan, A. Graham, Black ...",NaN,[''],NaN,"Digital Philosophy, 9/11 Articles, Race/Crime ...",NaN,0
167,26150,washingtonsblog.com,bias,http://www.washingtonsblog.com/2012/08/because...,I Was Silent When They Came for You … So There...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,I Will Stand Up For You If You Stand Up For Me,"Posted On, Sam Oconnel, President Ron Paul",NaN,[''],NaN,Podcasts from Boiling Frogs,NaN,0
190,29626,vdare.com,bias,http://www.vdare.com/posts/president-trump-sla...,In response to the Democrats closing the gover...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,President Trump Slams Democrats for Protecting...,NaN,NaN,"['vdare.com', 'blog']",In response to the Democrats closing the gover...,"Achievement Gap, GOP Share Of The White Vote, ...",NaN,0
191,29669,vdare.com,bias,http://www.vdare.com/posts/inauguration-day-ri...,I got the following press release from the D.C...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Inauguration Day Riot (J20)–New Trials (Of 59)...,NaN,NaN,"['vdare.com', 'blog']",I got the following press release from the D.C...,"Achievement Gap, GOP Share Of The White Vote, ...",NaN,0
196,30562,americanlookout.com,bias,http://americanlookout.com/media-critic-the-pr...,Howard Kurtz is the media critic for FOX News....,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Media Critic: “The Press Is Falling Into The P...,Mike Lachance,NaN,[''],NaN,"News, Donald Trump, Media, Media Bias, Politics",NaN,0


In [78]:
print(model.docvecs.most_similar(contTok[80]))
model.docvecs[5]

TypeError: '<' not supported between instances of 'str' and 'int'

In [32]:
df.content[40]

'Offers are getting into the six-figure range in pre-bidding for the NamesCon live auction\n\nHeadline: Bitcoin & Blockchain Searches Exceed Trump! Blockchain Stocks Are Next!\n\nIn a few weeks domain name investors from around the world will converge in Las Vegas for NamesCon, the largest event in the domain industry. One of the highlights of the conference is the live domain auction hosted by ROTD and pre-bidding has already started on NameJet, and like the title says, there are already six-figure bids.\n\nIf you’ve never been to a live domain auction I can tell you – it’s a rush that’s hard to explain, at least for a domain geek like me. Monte Cahn, one of the most respected domain industry veterans will be running the auction again this year and from what I’ve seen over the years, when Monte’s involved, a lot of the truly premium domains come out.\n\nSo I’m not incredibly surprised that we’re already starting to see six-figure bids come out in pre-bidding more than two weeks before